In [1]:
from langchain_community.document_loaders import PyPDFLoader




In [2]:
file_path = "./resource/PrivacyPolicyTemplate.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))


36


In [13]:
print(docs[35].page_content[0:1000])

# print(docs[0].metadata)


Y o u
c a n
c o n t a c t
t h e m
d i r e c t l y
r e g a r d i n g
o u r
p r o c e s s i n g
o f
y o u r
i n f o r m a t i o n ,
b y
e m a i l
a t
[ E m a i l
A d d r e s s ]
,
b y
v i s i t i n g
[ U K
R e p r e s e n t a t i v e
U R L ]
,
b y
p h o n e
a t
[ P h o n e
N u m b e r ]
,
o r
b y
p o s t
t o :
[ S t r e e t
A d d r e s s ]
[ C i t y ,
S t a t e
Z I P
C o d e ]
[ C o u n t r y ]
I f
y o u
h a v e
a n y
f u r t h e r
q u e s t i o n s
o r
c o m m e n t s ,
y o u
m a y
a l s o
c o n t a c t
u s
b y
p o s t
a t
t h e
f o l l o w i n g
c o r p o r a t e
a d d r e s s :
[ C o m p a n y
N a m e ]
[ S t r e e t
A d d r e s s ]
[ C i t y ,
S t a t e
Z I P
C o d e ]
[ C o u n t r y ]
P h o n e :
[ P h o n e
N u m b e r ]
1 8 .
H O W
C A N
Y O U
R E V I E W ,
U P D A T E ,
O R
D E L E T E
T H E
D A T A
W E
C O L L E C T
F R O M
Y O U ?
B a s e d
o n
t h e
a p p l i c a b l e
l a w s
o f
y o u r
c o u n t r y
o r
s t a t e
o f
r e s i d e n c e
i n
t h e
U S ,
y o u
m a y
h a v e
t 

In [4]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules["pysqlite3"]
import getpass
import os

os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq


llm = ChatGroq(model="llama3-8b-8192")

In [17]:


from langchain.embeddings import HuggingFaceEmbeddings
import faiss

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from uuid import uuid4



# text_splitter=CharacterTextSplitter(separator="\n",
#                                         chunk_size=1000,
#                                         chunk_overlap=200,
#                                         length_function=len)   
# chunks=text_splitter.split_text(raw_text)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                     model_kwargs={'device':'cpu'})
# Create vectors

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))


vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# vectorstore = faiss.FAISS.from_documents(docs, embeddings)

uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs, ids=uuids)


retriever = vector_store.as_retriever()



/home/vscode/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": " what is the policy under Canada?"})



results

{'input': ' what is the policy under Canada?',
 'context': [Document(metadata={'source': './resource/PrivacyPolicyTemplate.pdf', 'page': 25}, page_content='■\nF a c i l i t a t i o n\ni n\nt h e\nd e l i v e r y\no f\no u r\nS e r v i c e s\na n d\nt o\nr e s p o n d\nt o\ny o u r\ni n q u i r i e s .\nW e\nw i l l\nu s e\na n d\nr e t a i n\nt h e\nc o l l e c t e d\np e r s o n a l\ni n f o r m a t i o n\na s\nn e e d e d\nt o\np r o v i d e\nt h e\nS e r v i c e s\no r\nf o r :\n■\nC a t e g o r y\nA\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nB\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nC\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nD\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nE\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nF\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nG\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r y\nH\n-\n[ R e t e n t i o n\nP e r i o d ]\n■\nC a t e g o r 

In [22]:
print(results["context"][0].page_content)

print(results["context"][0].metadata)



■
F a c i l i t a t i o n
i n
t h e
d e l i v e r y
o f
o u r
S e r v i c e s
a n d
t o
r e s p o n d
t o
y o u r
i n q u i r i e s .
W e
w i l l
u s e
a n d
r e t a i n
t h e
c o l l e c t e d
p e r s o n a l
i n f o r m a t i o n
a s
n e e d e d
t o
p r o v i d e
t h e
S e r v i c e s
o r
f o r :
■
C a t e g o r y
A
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
B
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
C
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
D
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
E
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
F
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
G
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
H
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
I
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
J
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
K
-
[ R e t e n t i o n
P e r i o d ]
■
C a t e g o r y
L
-
[ R e t e n t i o n
P e r i o d ]
S o u r c e s
o f
P e r s o 

In [11]:
print(results["context"][0].page_content)




H o w e v e r ,
p l e a s e
n o t e
t h a t
t h i s
w i l l
n o t
a f f e c t
t h e
l a w f u l n e s s
o f
t h e
p r o c e s s i n g
b e f o r e
i t s
w i t h d r a w a l
n o r ,
w h e n
a p p l i c a b l e
l a w
a l l o w s ,
w i l l
i t
a f f e c t
t h e
p r o c e s s i n g
o f
y o u r
p e r s o n a l
i n f o r m a t i o n
c o n d u c t e d
i n
r e l i a n c e
o n
l a w f u l
p r o c e s s i n g
g r o u n d s
o t h e r
t h a n
c o n s e n t .
O p t i n g
o u t
o f
m a r k e t i n g
a n d
p r o m o t i o n a l
c o m m u n i c a t i o n s :
Y o u
c a n
u n s u b s c r i b e
f r o m
o u r
m a r k e t i n g
a n d
p r o m o t i o n a l
c o m m u n i c a t i o n s
a t
a n y
t i m e
b y
c l i c k i n g
o n
t h e
u n s u b s c r i b e
l i n k
i n
t h e
e m a i l s
t h a t
w e
s e n d ,
r e p l y i n g
“ S T O P ”
o r
“ U N S U B S C R I B E ”
t o
t h e
S M S
m e s s a g e s
t h a t
w e
s e n d ,
[ O t h e r ]
o r
b y
c o n t a c t i n g
u s
u s i n g
t h e
d e t a i l s
p r o v i d e d
i n


In [9]:
print(results["context"][0].page_content)




■
A d
N e t w o r k s
■
A ﬃ l i a t e
M a r k e t i n g
P r o g r a m s
■
D a t a
A n a l y t i c s
S e r v i c e s
■
R e t a r g e t i n g
P l a t f o r m s
■
S o c i a l
N e t w o r k s
■
U s e r
A c c o u n t
R e g i s t r a t i o n
&
A u t h e n t i c a t i o n
S e r v i c e s
■
[ O t h e r ]
T h e
c a t e g o r i e s
o f
t h i r d
p a r t i e s
t o
w h o m
w e
s h a r e d
p e r s o n a l
i n f o r m a t i o n
w i t h
a r e :
■
A d
N e t w o r k s
■
A ﬃ l i a t e
M a r k e t i n g
P r o g r a m s
■
D a t a
A n a l y t i c s
S e r v i c e s
■
R e t a r g e t i n g
P l a t f o r m s
■
S o c i a l
N e t w o r k s
■
U s e r
A c c o u n t
R e g i s t r a t i o n
&
A u t h e n t i c a t i o n
S e r v i c e s
■
[ O t h e r ]
[
N O T I C E
:
T h i s
w e b s i t e
m a y
s e l l
y o u r
s e n s i t i v e
p e r s o n a l
i n f o r m a t i o n . ]
[
N O T I C E
:
T h i s
w e b s i t e
m a y
s e l l
y o u r
b i o m e t r i c
p e r s o n a l
i n f o r m a t i o n . ] ]
Y o u r
R i g h t s
Y o u
